In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
data_df = pd.read_pickle('data_clean.pickle')

In [3]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = TfidfVectorizer()
data_cvn = cvn.fit_transform(data_df.text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
data_dtmn.columns = data_df.index
data_dtmn

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
aa,0.000000,0.000105,0.0,0.000084,0.000090,0.0,0.0,0.000000,0.000127,0.000652,0.000039,0.0,0.0
aaa,0.000354,0.000000,0.0,0.000037,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000052,0.0,0.0
aaaaa,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000548,0.000000,0.000000,0.000000,0.0,0.0
aaaaaaaa,0.000000,0.000000,0.0,0.000016,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
aaaaaaaaaaaa,0.000000,0.000000,0.0,0.000014,0.000067,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝙄𝙏,0.000000,0.000000,0.0,0.000016,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
𝙏𝙃𝙀,0.000000,0.000000,0.0,0.000033,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
𝙔𝙀𝙎,0.000000,0.000000,0.0,0.000016,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
𝚃𝚑𝚒𝚜,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000548,0.000000,0.000000,0.000000,0.0,0.0


In [4]:
data_dtmn['index'] = data_dtmn.index
for i, row in enumerate(data_dtmn['index']):
    candidate = re.match('a{2,}', str(row))
    if candidate != None:
        data_dtmn.drop(row, inplace=True)
    if row == 'ababababba':
        data_dtmn.drop(row, inplace=True)
    if row == 'wa':
        data_dtmn.drop(row, inplace=True)       
    if row == 'ha':
        data_dtmn.drop(row, inplace=True)    
data_dtmn.drop(columns=['index'], inplace=True)
data_dtmn

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
ab,0.000147,0.000280,0.0,0.000060,0.000000,0.0,0.0,0.000000,0.000234,0.0,0.000054,0.000058,0.000262
aba,0.000000,0.000031,0.0,0.000028,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
abacavir,0.000000,0.000018,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
aback,0.000000,0.000042,0.0,0.000019,0.000045,0.0,0.0,0.000000,0.000000,0.0,0.000019,0.000031,0.000107
abagails,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000221,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝙄𝙏,0.000000,0.000000,0.0,0.000016,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
𝙏𝙃𝙀,0.000000,0.000000,0.0,0.000033,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
𝙔𝙀𝙎,0.000000,0.000000,0.0,0.000016,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
𝚃𝚑𝚒𝚜,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000548,0.000000,0.0,0.000000,0.000000,0.000000


In [5]:
data_dtmn.transpose()

,ab,aba,abacavir,aback,abagails,abalone,abandon,abandoned,abandonedranging,abandoning,...,𝘽𝙊𝘽,𝘽𝙐𝙄𝙇𝘿𝙀𝙍,𝘾𝘼𝙉,𝙁𝙄𝙓,𝙃𝙀,𝙄𝙏,𝙏𝙃𝙀,𝙔𝙀𝙎,𝚃𝚑𝚒𝚜,𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎
wsb,0.000147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000042,0.000094,0.000000,0.000091,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
science,0.000280,0.000031,0.000018,0.000042,0.000000,0.000000,0.000091,0.000136,0.000000,0.000018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ama,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000427,0.000639,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
askreddit,0.000060,0.000028,0.000000,0.000019,0.000000,0.000000,0.000118,0.000149,0.000016,0.000072,...,0.000033,0.000033,0.000033,0.000016,0.000033,0.000016,0.000033,0.000016,0.000000,0.000000
worldnews,0.000000,0.000000,0.000000,0.000045,0.000000,0.000000,0.000142,0.000212,0.000000,0.000076,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
funny,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000185,0.000277,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dankmemes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000283,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
memes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000186,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000548,0.000548
nosleep,0.000234,0.000000,0.000000,0.000000,0.000221,0.000000,0.000000,0.000525,0.000000,0.000108,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
psychology,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000192,0.000000,0.000277,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
from gensim import matutils, models
import scipy.sparse

In [7]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [8]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=10, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 21:56:54: using symmetric alpha at 0.1
INFO - 21:56:54: using symmetric eta at 0.1
INFO - 21:56:54: using serial LDA version on this node
INFO - 21:56:55: running online LDA training, 10 topics, 50 passes over the supplied corpus of 112138 documents, updating every 30000 documents, evaluating every ~112138 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 21:56:55: training LDA model using 15 processes
INFO - 21:56:55: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/112138, outstanding queue size 1
INFO - 21:56:55: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/112138, outstanding queue size 2
INFO - 21:56:55: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/112138, outstanding queue size 3
INFO - 21:56:55: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/112138, outstanding queue size 4
INFO - 21:56:55: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/112138, outstanding queue size 5
INF

INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #40 = documents up to #82000/112138, outstanding queue size 5
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #41 = documents up to #84000/112138, outstanding queue size 6
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #42 = documents up to #86000/112138, outstanding queue size 6
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #43 = documents up to #88000/112138, outstanding queue size 5
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #44 = documents up to #90000/112138, outstanding queue size 4
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #45 = documents up to #92000/112138, outstanding queue size 3
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #46 = documents up to #94000/112138, outstanding queue size 3
INFO - 21:56:59: PROGRESS: pass 0, dispatched chunk #47 = documents up to #96000/112138, outstanding queue size 4
INFO - 21:56:59: PROGRESS: pass 0, dispatched chunk #48 = documents up to #98000/112138,

INFO - 21:57:01: topic #2 (0.100): 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaarrrrggg" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa"
INFO - 21:57:01: topic #1 (0.100): 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaa" + 0.000*"aaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaah"
INFO - 21:57:01: topic diff=0.000062, rho=0.131228
INFO - 21:57:01: PROGRESS: pass 1, dispatched chunk #19 = documents up to #40000/112138, outstanding queue size 4
INFO - 21:57:02: PROGRESS: pass 1, dispatched chunk #20 = documents up to #42000/112138, outstanding queue size 3
INFO - 21:57:02: PROGRESS: pass 1, dispatched chunk #21 = documents up to #44000/112138, outstanding queue size 3
INFO - 21:

INFO - 21:57:05: merging changes from 18138 documents into a model of 112138 documents
INFO - 21:57:05: topic #7 (0.100): 0.001*"aa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaarrrrggg" + 0.001*"aaaaa" + 0.001*"aaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
INFO - 21:57:05: topic #0 (0.100): 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaarrrrggg"
INFO - 21:57:05: topic #3 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaarrrrggg" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa"
INFO - 21:57:0

INFO - 21:57:08: topic #2 (0.100): 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaarrrrggg" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa"
INFO - 21:57:08: topic diff=0.000049, rho=0.130113
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #34 = documents up to #70000/112138, outstanding queue size 4
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #35 = documents up to #72000/112138, outstanding queue size 2
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #36 = documents up to #74000/112138, outstanding queue size 3
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #37 = documents up to #76000/112138, outstanding queue size 4
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #38 = documents up to #78000/112138, outstanding queue size 5
INFO - 21:57:08: PROGRESS: pass 2, dispatched chunk #39 = documents up to #

...

INFO - 22:00:43: PROGRESS: pass 48, dispatched chunk #18 = documents up to #38000/112138, outstanding queue size 5
INFO - 22:00:43: merging changes from 30000 documents into a model of 112138 documents
INFO - 22:00:43: topic #7 (0.100): 0.001*"aa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaarrrrggg" + 0.001*"aaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaall" + 0.001*"aaaaaaaaaaaa"
INFO - 22:00:43: topic #1 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:43: topic #3 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaa

INFO - 22:00:46: topic diff=0.000046, rho=0.097558
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #50 = documents up to #102000/112138, outstanding queue size 6
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #51 = documents up to #104000/112138, outstanding queue size 4
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #52 = documents up to #106000/112138, outstanding queue size 5
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #53 = documents up to #108000/112138, outstanding queue size 4
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #54 = documents up to #110000/112138, outstanding queue size 4
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #55 = documents up to #112000/112138, outstanding queue size 4
INFO - 22:00:46: PROGRESS: pass 48, dispatched chunk #56 = documents up to #112138/112138, outstanding queue size 4
INFO - 22:00:46: merging changes from 22138 documents into a model of 112138 documents
INFO - 22:00:46: topic #9 (0.100): 0.000*"aaaaaaaa

INFO - 22:00:49: topic #8 (0.100): 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:49: topic #7 (0.100): 0.001*"aa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaarrrrggg" + 0.001*"aaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaall" + 0.001*"aaaaaaaaaaaa"
INFO - 22:00:49: topic #2 (0.100): 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:49: topic #3 (0.100): 0.000*"aa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaah" + 0.000*"aaaaaaaaaaa

INFO - 22:00:51: topic #4 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:51: topic #5 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:51: topic #6 (0.100): 0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"
INFO - 22:00:51: topic #7 (0.100): 0.001*"aa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.001*"aaaa

[(0,
  '0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"'),
 (1,
  '0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"'),
 (2,
  '0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaall" + 0.000*"aaaaaaaaaaaa"'),
 (3,
  '0.000*"aaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aa" + 0.000*"aaaaaaaaah" + 0.000*"aaa" + 0.000*"aaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaa" + 0.000*"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" + 0.00